<a href="https://colab.research.google.com/github/wcj365/pandas-grouping/blob/master/groupby_melt_pivot_crosstab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pandas分组统计：groupby，melt，pivot_table，crosstab的用法
- groupby: 分组
- melt: 宽表转长表
- pivot_table: 长表转宽表，数据透视表
- crosstab: 交叉表 / 列联表，主要用于分组频数统计

Reference: 
- https://www.cnblogs.com/iupoint/p/11050887.html
- https://www.pypandas.cn/docs/user_guide/reshaping.html#reshaping-by-pivoting-dataframe-objects

In [0]:
import numpy as np
import pandas as pd
 
df = pd.DataFrame({'key1':['a','a','b','b','a'],
                   'key2':['one','two','one','two','one'],
                   'data1':np.random.randn(5),
                   'data2':np.random.randn(5)
                   })

df


,key1,key2,data1,data2
0,a,one,0.636798,0.863032
1,a,two,-0.110265,-0.627385
2,b,one,-2.287199,-0.701121
3,b,two,-0.830498,1.223054
4,a,one,0.741648,0.187144


In [0]:
g1 = df.groupby("key1")
g1.groups

{'a': Int64Index([0, 1, 4], dtype='int64'),
 'b': Int64Index([2, 3], dtype='int64')}

In [0]:
g1.get_group("a")

,key1,key2,data1,data2
0,a,one,0.636798,0.863032
1,a,two,-0.110265,-0.627385
4,a,one,0.741648,0.187144


In [0]:
[x for x in g1]

[('a',   key1 key2     data1     data2
  0    a  one  0.636798  0.863032
  1    a  two -0.110265 -0.627385
  4    a  one  0.741648  0.187144), ('b',   key1 key2     data1     data2
  2    b  one -2.287199 -0.701121
  3    b  two -0.830498  1.223054)]

In [0]:
g1.sum()

,data1,data2
key1,,
a,1.268181,0.422791
b,-3.117697,0.521933


In [0]:
g1.size()

key1
a    3
b    2
dtype: int64

In [0]:
g2 = df.groupby(["key1","key2"])
g2.size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [0]:
g2.count()

data1  data2
key1 key2              
a    one       2      2
     two       1      1
b    one       1      1
     two       1      1

In [0]:
g1['data1','data2'].agg(['mean','sum'])  #作用于所有列

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


data1               data2          
          mean       sum      mean       sum
key1                                        
a     0.422727  1.268181  0.140930  0.422791
b    -1.558849 -3.117697  0.260966  0.521933

In [0]:
g2['data1','data2'].apply(lambda x: pd.Series([x.shape[0], x['data1'].mean(), x['data2'].sum()],
       index=['counts', 'key1_mean', 'key2_sum']))  #作用于指定列

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


counts  key1_mean  key2_sum
key1 key2                             
a    one      2.0   0.689223  1.050176
     two      1.0  -0.110265 -0.627385
b    one      1.0  -2.287199 -0.701121
     two      1.0  -0.830498  1.223054

In [0]:
# ####################
''' melt 用法 -- 宽表转长表 '''
pd.melt(df, id_vars=['key1', 'key2'], value_vars=['data1', 'data2'], var_name='var', value_name='value')  #col_level

,key1,key2,var,value
0,a,one,data1,0.636798
1,a,two,data1,-0.110265
2,b,one,data1,-2.287199
3,b,two,data1,-0.830498
4,a,one,data1,0.741648
5,a,one,data2,0.863032
6,a,two,data2,-0.627385
7,b,one,data2,-0.701121
8,b,two,data2,1.223054
9,a,one,data2,0.187144


In [0]:
''' crosstab 用法 -- 列联表（count） '''
# Apply to categorical variables only

pd.crosstab(df.key1, df.key2, margins=True)

key2,one,two,All
key1,,,
a,2,1,3
b,1,1,2
All,3,2,5


In [0]:
''' pivot_table 用法 -- 长表转宽表 '''
# pd.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None,
# 　　　　　　　　dropna=True, margins=False, margins_name='ALL')  #aggfunc={'d':np.sum, 'e':np.max}
pd.pivot_table(df, index='key1', columns='key2')

data1               data2          
key2       one       two       one       two
key1                                        
a     0.689223 -0.110265  0.525088 -0.627385
b    -2.287199 -0.830498 -0.701121  1.223054

In [0]:
df.pivot_table(['data1'], index='key1', columns='key2', fill_value=0)  #['data1']

data1          
key2       one       two
key1                    
a     0.689223 -0.110265
b    -2.287199 -0.830498